## Columns to be removed from training/validation

# Load Tensorflow and check GPU availability

In [2]:
# -*- coding: utf-8 -*-
import torch
from core.loader import Loader
from models.model_wrapper import ModelWrapper


from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from core.loader import Segmenter


2025-05-13 12:17:47.976463: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-13 12:17:47.976495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-13 12:17:47.977510: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-13 12:17:47.983175: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 12:17:48.787941: W tensorflow/compiler/tf2

2025-05-13 12:17:49.607330: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
/home/poli/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
2025-05-13 12:17:49.607353: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: poli
2025-05-13 12:17:49.607358: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: poli
2025-05-13 12:17:49.607392: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostic

# Load input datasets

In [3]:
benign_dataset_filenames = [
    'parkets/benign/benign_2312_anonymized_HTML.parquet', 
    'parkets/benign/umbrella_benign_FINISHED_HTML.parquet'
        
]
malicious_dataset_filenames = [
    'parkets/malware_2406_strict_HTML.parquet'
]


# print me number of domains from each separate dataset

# CONFIGURATION

benign_label = "benign"
malicious_label = "malware"

class_map = {benign_label: 0, malicious_label: 1}
# print labels from malicious datasets

loader = Loader(benign_dataset_filenames, malicious_dataset_filenames, benign_label=benign_label, malicious_label=malicious_label, subsample=1.0)
df = loader.load()



# split into 3 stages 

In [12]:
from core.loader import Segmenter

# Define the aggregates that needs to be created

aggregates = [
    ["lex_", "dns_", "ip_", "tls_", "geo_", "rdap_"],
]

segmenter = Segmenter(df)
segmenter.create_base_subsets() # create base subsets
segmenter.create_aggregated_subsets(aggregates)
subset_dfs = segmenter.get_aggregated_subsets()

/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

# Define the NN model

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, BatchNormalization, ReLU, Dropout,
    Multiply, Add
)
from tensorflow.keras.activations import sigmoid


ARCH_NAME = "attention_tls"
LR = 0.0023  # Slightly lower for better convergence
VERSION = "v1.1"

def build_tls_classifier(input_dim):
    inputs = Input(shape=(input_dim,), name="input")

    # Attention mechanism
    attention_weights = Dense(input_dim, activation="sigmoid")(inputs)
    attention_applied = Multiply(name="attention_applied")([inputs, attention_weights])

    # Input projection
    x = Dense(512, name="dense_512")(attention_applied)
    x = BatchNormalization(name="bn_512")(x)
    x = ReLU(name="relu_512")(x)

    # First hidden layer
    x = Dense(256, name="dense_256")(x)
    x = BatchNormalization(name="bn_256")(x)
    x = ReLU(name="relu_256")(x)
    x = Dropout(0.3, name="dropout_256")(x)

    # Second hidden layer
    x = Dense(128, name="dense_128")(x)
    x = BatchNormalization(name="bn_128")(x)
    x = ReLU(name="relu_128")(x)
    x = Dropout(0.3, name="dropout_128")(x)

    # Skip connection from input
    skip = Dense(128, activation="relu", name="skip_connection")(inputs)

    # Add skip connection
    x = Add(name="add_skip")([x, skip])

    # Output layer
    outputs = Dense(1, activation="sigmoid", name="output")(x)

    model = Model(inputs=inputs, outputs=outputs, name="tls_classifier")
    return model

### Save subsets

### Load subsets

In [ ]:
import gc
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from models.model_wrapper import ModelWrapper
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


model_histories = []

# make sure TF only allocates as much GPU memory as it needs
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

wrapper = ModelWrapper()


for prefix, subset_df in subset_dfs.items():


    print(f"\n🚀 Training Feedforward NN on '{prefix}' features…")
    labels   = subset_df['label'].map(class_map)
    features = loader.scale(subset_df.drop('label', axis=1), stage=3, model=ARCH_NAME)

    X_train, X_test, Y_train, Y_test = train_test_split(
        features, labels,
        test_size=0.2, random_state=42,
        shuffle=True, stratify=labels
    )

    model = build_tls_classifier(X_train.shape[1])
    model.compile(
        optimizer="adam",
        loss='binary_crossentropy',
        metrics=['Precision', 'Recall', 'AUC']
    )

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    history = model.fit(
        X_train, Y_train,
        batch_size=512,
        epochs=15,
        validation_data=(X_test, Y_test),
        class_weight={0: 1.0, 1: 0.7},
        callbacks=[early_stopping]
    )
    

    model_histories.append({"model_name": prefix,
                      "history": history})
    

    wrapper.save(model,
                 arch_name=ARCH_NAME,
                 label=malicious_label,
                 prefix=prefix,
                 version=VERSION)

    # ---- here’s the magic ----
    K.clear_session()    # drops the entire TF graph + variables
    del model           # remove the Python reference
    del history         # free training history
    del X_train, X_test, Y_train, Y_test, features, labels
    gc.collect()        # ask Python to free unreferenced memory


🚀 Training Feedforward NN on 'lex_+dns_+ip_+tls_+geo_+rdap_agg' features…


2025-05-05 13:17:52.629969: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1104715392 exceeds 10% of free system memory.


Epoch 1/15
1533/1533 [==============================] - 12s 6ms/step - loss: 0.0516 - precision: 0.9597 - recall: 0.9018 - auc: 0.9930 - val_loss: 0.0444 - val_precision: 0.9743 - val_recall: 0.9338 - val_auc: 0.9960
Epoch 2/15
1533/1533 [==============================] - 8s 5ms/step - loss: 0.0331 - precision: 0.9762 - recall: 0.9391 - auc: 0.9962 - val_loss: 0.0393 - val_precision: 0.9836 - val_recall: 0.9370 - val_auc: 0.9965
Epoch 3/15
1533/1533 [==============================] - 8s 5ms/step - loss: 0.0288 - precision: 0.9796 - recall: 0.9468 - auc: 0.9969 - val_loss: 0.0356 - val_precision: 0.9759 - val_recall: 0.9524 - val_auc: 0.9969
Epoch 4/15
1533/1533 [==============================] - 8s 5ms/step - loss: 0.0257 - precision: 0.9824 - recall: 0.9524 - auc: 0.9973 - val_loss: 0.0368 - val_precision: 0.9852 - val_recall: 0.9423 - val_auc: 0.9963
Epoch 5/15
1533/1533 [==============================] - 8s 6ms/step - loss: 0.0236 - precision: 0.9836 - recall: 0.9561 - auc: 0.9975 -

: 

In [ ]:
import matplotlib.pyplot as plt

# Assume model_histories is a list of dicts, each with keys "model_name" and "history"
def get_metric(history, metric_name):
    if metric_name in history:
        return history[metric_name]
    for suffix in ["_12", "_2"]:
        if f"{metric_name}{suffix}" in history:
            return history[f"{metric_name}{suffix}"]
    raise KeyError(f"Metric {metric_name} not found in history.")

for model_entry in model_histories:
    name = model_entry["model_name"]
    history = model_entry["history"].history  # Keras history object

    epoch_losses = get_metric(history, 'loss')
    epoch_val_losses = get_metric(history, 'val_loss')
    epoch_accuracies = get_metric(history, 'auc')
    epoch_val_accuracies = get_metric(history, 'val_auc')
    epoch_precisions = get_metric(history, 'precision')
    epoch_val_precisions = get_metric(history, 'val_precision')
    epoch_recalls = get_metric(history, 'recall')
    epoch_val_recalls = get_metric(history, 'val_recall')

    # Calculate F1 scores
    def safe_f1(p, r):
        return 2 * (p * r) / (p + r) if (p + r) > 0 else 0

    epoch_f1s = [safe_f1(p, r) for p, r in zip(epoch_precisions, epoch_recalls)]
    epoch_val_f1s = [safe_f1(p, r) for p, r in zip(epoch_val_precisions, epoch_val_recalls)]

    # Plot
    plt.figure(figsize=(18, 10))
    plt.subplot(2, 3, 1)
    plt.plot(epoch_losses, 'b--o', label='Training Loss')
    plt.plot(epoch_val_losses, 'r--o', label='Validation Loss')
    plt.title('Loss'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)

    plt.subplot(2, 3, 2)
    plt.plot(epoch_accuracies, '--o', label='Training AUC', color='#ff7f0e')
    plt.plot(epoch_val_accuracies, 'r--o', label='Validation AUC')
    plt.title('AUC'); plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.legend(); plt.grid(True)

    plt.subplot(2, 3, 3)
    plt.plot(epoch_precisions, 'g--o', label='Training Precision')
    plt.plot(epoch_val_precisions, 'r--o', label='Validation Precision')
    plt.title('Precision'); plt.xlabel('Epoch'); plt.ylabel('Precision'); plt.legend(); plt.grid(True)

    plt.subplot(2, 3, 4)
    plt.plot(epoch_recalls, 'c--o', label='Training Recall')
    plt.plot(epoch_val_recalls, 'r--o', label='Validation Recall')
    plt.title('Recall'); plt.xlabel('Epoch'); plt.ylabel('Recall'); plt.legend(); plt.grid(True)

    plt.subplot(2, 3, 5)
    plt.plot(epoch_f1s, 'm--o', label='Training F1')
    plt.plot(epoch_val_f1s, 'r--o', label='Validation F1')
    plt.title('F1 Score'); plt.xlabel('Epoch'); plt.ylabel('F1'); plt.legend(); plt.grid(True)

    plt.suptitle(f"Training Progress - {name}", fontsize=16, y=1.02)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f'./figures/training_{ARCH_NAME}_{name}_{VERSION}.png', dpi=500, bbox_inches='tight', pad_inches=0.5)
    plt.close()
